In [1]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2

In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # get the dimensions of the frame and construct its blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 
                                 1.0, 
                                 (300, 300),
                                 (104.0, 177.0, 123.0))

    # obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # initialize
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence of the detection
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            # compute the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # check for boundary constraints
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI and preprocessing it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        preds = maskNet.predict(faces)

    return (locs, preds)



In [3]:
# load the face detector model
path = 'Face_Mask_Detection/'
prototxtPath = path + 'face_detector/deploy.prototxt'
weightsPath = path + 'face_detector/res10_300x300_ssd_iter_140000.caffemodel'
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [4]:
# load the face mask detector model
maskNet = load_model(path+'model.model')

In [5]:
vs = VideoStream(src=0).start()
time.sleep(2.0)

In [6]:
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=800)

    # detect faces in the frame and determine if they are wearing a face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output frame
        cv2.putText(frame, 
                    label, 
                    (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.45, 
                    color, 
                    2)
        cv2.rectangle(frame, 
                      (startX, startY), 
                      (endX, endY), 
                      color, 
                      2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

In [7]:
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()